In [ ]:
RDATEC = 202012 #/*current quarter*/
RDATEP = 202009 #/*last quarter*/

RDATE2 = '2020-12' #/*current quarter*/

RDATE3C = 20201231 #/*current quarter*/
RDATE3P = 20200930 #/*last quarter*/

import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan

MASTER_LIST_CB_C = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\01_Corporate_Financing\04_Dataset\\'+str(RDATE3C)+'\MIB_CB_BNM_SUMM_'+str(RDATE3C)+'.xlsx', sheet_name='MASTER')
MASTER_LIST_CB_P = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\01_Corporate_Financing\04_Dataset\\'+str(RDATE3P)+'\MIB_CB_BNM_SUMM_'+str(RDATE3P)+'.xlsx', sheet_name='MASTER')

mib_bb_cusfinal_C = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\01_Portfolio\\'+str(RDATE3C)+'\BB2_SUMMARY_'+str(RDATE3C)+'.xlsx', sheet_name='CUSBASE2')
mib_bb_cusfinal_P = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\01_Portfolio\\'+str(RDATE3P)+'\BB2_SUMMARY_'+str(RDATE3P)+'.xlsx', sheet_name='CUSBASE2')


#/************************ CB***************************/

MASTER_LIST_CB_C.CUS_STATUS = MASTER_LIST_CB_C.CUS_STATUS.str.strip()
MASTER_LIST_CB_P.CUS_STATUS = MASTER_LIST_CB_P.CUS_STATUS.str.strip()

def Risk_Category(CUS_STATUS,Risk_Category):
    if CUS_STATUS == 'IMPAIRED':
        return 'IF'
    else:
        return Risk_Category
MASTER_LIST_CB_C['Risk_Category'] = MASTER_LIST_CB_C.apply(lambda x: Risk_Category(x['CUS_STATUS'], x['Risk_Category']), axis=1)    

def Risk_Category_P(CUS_STATUS,Risk_Category):
    if CUS_STATUS == 'IMPAIRED':
        return 'IF'
    else:
        return Risk_Category
MASTER_LIST_CB_P['Risk_Category_P'] = MASTER_LIST_CB_P.apply(lambda x: Risk_Category_P(x['CUS_STATUS'], x['Risk_Category']), axis=1)    

CB01 = MASTER_LIST_CB_C.merge(MASTER_LIST_CB_P[['CUS_NO','Risk_Category_P']], on='CUS_NO', how='left', indicator = True)

CB01.loc[CB01._merge=='left_only','Risk_Category_P']='NEW'

CB01_1 = CB01[['CUS_NO','Name','CUS_LIMIT_MIL','CUS_BAL_TL_MIL','CUS_BAL_TL_OD','CUS_BAL_STRC_MIL','CUS_BAL_TB_MIL','CUS_BAL_RTO_MIL','CUS_BAL_MIL','Risk_Category','COMBINED_RATING','SECTOR_GROUP','Nature_Of_Business','M_Msic_Code','ISC_PARENT','SEGMENT','CUS_STATUS','RR_STATUS','CUST_GROUP','SCORECARDS','Risk_Category_P']]

#/************************ BB ***************************/

mib_bb_cusfinal_C.CUS_STATUS = mib_bb_cusfinal_C.CUS_STATUS.str.strip()
mib_bb_cusfinal_P.CUS_STATUS = mib_bb_cusfinal_P.CUS_STATUS.str.strip()

def RISK_CAT(CUS_STATUS,RISK_CAT):
    if CUS_STATUS == 'IMPAIRED':
        return 'IF'
    else:
        return RISK_CAT
mib_bb_cusfinal_C['RISK_CAT'] = mib_bb_cusfinal_C.apply(lambda x: RISK_CAT(x['CUS_STATUS'], x['RISK_CAT']), axis=1)    

def RISK_CAT_P(CUS_STATUS,RISK_CAT):
    if CUS_STATUS == 'IMPAIRED':
        return 'IF'
    else:
        return RISK_CAT
mib_bb_cusfinal_P['RISK_CAT_P'] = mib_bb_cusfinal_P.apply(lambda x: RISK_CAT_P(x['CUS_STATUS'], x['RISK_CAT']), axis=1)    

BB01 = mib_bb_cusfinal_C.merge(mib_bb_cusfinal_P[['GCIF','RISK_CAT_P']], on='GCIF', how='left', indicator = True)

BB01.loc[BB01._merge=='left_only','RISK_CAT_P']='NEW'

BB01_1 = BB01[['GCIF','MBB_Islamic_Ind','Borrower_Name','BRR','FRR','ISC_NOB','ISC_PARENT','SECTOR_CODE','Broad_Sector','REGION','BC','RISK_CAT','COLLATERAL_TYPE','CUS_BAL','NOC','CUS_OSM','RR_TYPE','WATCHLIST_FLAG','SMA_FLAG','IMPAIRED_FLAG','EWS_FLAG','RESCHEDULED_FLAG','RESTRUCTURED_FLAG','R_N_R_FLAG','CUS_STATUS','RR_STATUS','RISK_CAT_P']]

writer = pd.ExcelWriter(r'T:\\MIB Risk Management\\Credit Risk Analytics - Historical\\01_Reporting\\05_Portfolio_Analysis\\00_Credit_Risk_Profile\\'+str(RDATE2)+'\\rating migration\\NonRetail\\RM_CBBB_Q420(python).xlsx', engine='xlsxwriter')

CB01_1.to_excel(writer, sheet_name='CB', index = False)
BB01_1.to_excel(writer, sheet_name='BB', index = False)

writer.save()
